# Tool calling example

In [ ]:
from openai import OpenAI
from datetime import datetime, timedelta
import requests
import json
import os


# --------------------- DEFINE SOME TOY FUNCTIONS TO CALL IN THIS DEMO

def get_current_date_time() -> str:
    now = datetime.now()
    return now.strftime("%A, %B %d, %Y %H:%M:%S")


def get_current_exchange_rate(currency: str, new_currency: str) -> float | None:
    """Get latest exchange rate for currency pair"""
    return 2.0

def get_historical_exchange_rate(currency: str, new_currency: str, date: str) -> float | None:
    """Get historical exchange rate for currency pair on given date"""
    return 3.0

def get_bio_of_person(person_id: str) -> str:
    """Get a description of current knowledge of a person based on the persons 10 digit CPR number provided as a string in the format DDMMYYYY-WXYZ or DDMMYYYYWXYZ"""
    if person_id == "060292-9981":
        return "This person (060292-9981) is named Peter. He is a member of Thisted Mejeriforening"

    elif person_id == "3010293-2154":
        return "This person (3010293-2154) is named Allan. He is a member of Skjern Skakklub"
    else:
        return "Your task: Tell the user that we do not have a bio on this person and encourage them to check if they entered the right CPR number and tell them that they can try a different CPR number."
        


    
# --------------- DEFINE JSON THAT TELLS THE LLM WHAT TOOLS ARE AVAILABLE

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_date_time",
            "description": "Gets the current system time and formats it as a string.",
            "parameters": {"type": "object", "properties": {}},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_bio_of_person",
            "description": "Everytime you encounter a message with a digital that follows the format DDMMYYYY-WXYZ or DDMMYYYYWXYZ, analyze whether to call this tool. The tool gives you a description of our current knowledge of a person based on the persons 10 digit CPR number aka 'personnummer' or 'pnr' provided as a string in the format DDMMYYYY-WXYZ or DDMMYYYYWXYZ.",
            "parameters": {
                "type": "object",
                "properties": {
                    "person_id": {"type": "string"},
                },
                "required": ["person_id"],
            },        
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_exchange_rate",
            "description": "Get the latest exchange rate for the currency pair.",
            "parameters": {
                "type": "object",
                "properties": {
                    "currency": {"type": "string"},
                    "new_currency": {"type": "string"},
                },
                "required": ["currency", "new_currency"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_historical_exchange_rate",
            "description": "Get the historical exchange rate for the currency pair on a specific date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "currency": {"type": "string"},
                    "new_currency": {"type": "string"},
                    "date": {"type": "string"},
                },
                "required": ["currency", "new_currency", "date"],
            },
        },
    },
]


# --- Local mapping from tool name to function 
FUNCTION_MAP = {
    "get_current_date_time": get_current_date_time,
    "get_current_exchange_rate": get_current_exchange_rate,
    "get_historical_exchange_rate": get_historical_exchange_rate,
    "get_bio_of_person" : get_bio_of_person,
}


def run_example(prompt: str, model_id):
    """Send a message, handle a tool call if the model makes one."""
    
    print(f"\n=== USER: {prompt} ===")

    # Call the API
    response = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        tools=tools,
    )

    message = response.choices[0].message

    # Step 1: Check if the model wants to call a tool
    if message.tool_calls:
        for tool_call in message.tool_calls:
            fn_name = tool_call.function.name
            fn_args = json.loads(tool_call.function.arguments)
            print(f"? Model requested tool: {fn_name}({fn_args})")

            # Step 2: Execute the tool locally
            if fn_name in FUNCTION_MAP:
                
                result = FUNCTION_MAP[fn_name](**fn_args)
                print(f"? Tool result: {result}")

                # Step 3: Send result back to model for final answer
                follow_up = client.chat.completions.create(
                    model=model_id,
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt},
                        message,
                        {
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "content": json.dumps(result),
                        },
                    ],
                )

                final_msg = follow_up.choices[0].message
                print("? Final Assistant Response:\n\n", final_msg.content)

    else:
        print("? Model replied directly:", message.content)

endpoint = "https://some.address.local/api/v1/llm-general-purpose-secondary/v1"

model_id = "google/gemma-3-27b-it"

# --- Initialize client ---
client = OpenAI(
    base_url=endpoint,
    api_key="no-key"
)



# --- Run some examples ---

# generic examples from the internet
run_example("What time is it right now?", model_id=model_id)
run_example("What was the USD to JPY exchange rate on 2023-05-01?", model_id=model_id)

# Show that the model can call a fictional database with info about persons
run_example("Hvilke organisationer er pnr 060292-9981 tilknyttet?", model_id=model_id)
run_example("Hvad ved vi om 060292-9981?", model_id=model_id)
run_example("Hvilke organisationer er person med cpr nr 121123-9981 tilknyttet?", model_id=model_id) # Vi simulerer at dette ID ikke virker
run_example("Fortæl mig om person 3010293-2154?", model_id=model_id) 


